In [2]:
import pandas as pd 
import numpy as np

In [24]:
def onehot_name(df):
    mrs = '|'.join(['Mrs\.','Mme\.', 'Ms\.', 'Dr\.', 'Lady\.', 'Countess\.'])    
    mr = '|'.join(['Mr\.','Don\.','Rev\.','Dr\.', 'Major\.', 'Sir\.', 'Col\.', 'Capt\.', 'Jonkheer\.'])
    miss = '|'.join(['Miss\.', 'Mlle\.'])
    return df.assign(Name = lambda x : x['name'].str.replace('\(.*\)', '', regex=True),
                     Mr =  lambda x : x['name'].str.contains(mr).astype(int) ,
                     Miss = lambda x : x['name'].str.contains(miss).astype(int),
                     Mrs = lambda x : x['name'].str.contains(mrs).astype(int),
                     Master = lambda x : (x['name'].str.find('Master.') > -1).astype(int))

In [28]:
def read_data(file):
    """
    
    """
    df = (pd.read_csv(file, dtype = {
                            "PassengerId":np.int32,
                            "Name": "object",
                            "Pclass":np.int,
                            "Survived":np.int32,
                            "Sex":"object",
                            "Age":np.float,
                            "SibSp":np.int,
                            "Embarked":"object",
                            "Cabin":"object",
                            "Fare":np.float64,
                        },)
            .rename(columns=str.lower)
            .set_index("passengerid")
            .pipe(onehot_name)
            #.drop(['Name','Ticket','Cabin'],axis=1)
        )
    return df
    

In [29]:
data = read_data("../data/train.csv")

## Family size 

In [34]:
data = (data.assign(family_size = data.sibsp + data.parch+1,
                    is_alone = lambda x: x.family_size < 2,
                    calc_fare = lambda x: x.fare/x.family_size)
            .astype({'is_alone':int}))


In [36]:
data.head()

,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,Name,Mr,Miss,Mrs,Master,family_size,is_alone,calc_fare
passengerid,,,,,,,,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,"Braund, Mr. Owen Harris",1,0,0,0,2,0,3.62500
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,"Cumings, Mrs. John Bradley",0,0,1,0,2,0,35.64165
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,"Heikkinen, Miss. Laina",0,1,0,0,1,1,7.92500
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,"Futrelle, Mrs. Jacques Heath",0,0,1,0,2,0,26.55000
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,"Allen, Mr. William Henry",1,0,0,0,1,1,8.05000
